In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import ipyvolume as ipv



from tyssue import Epithelium, RNRGeometry
from tyssue.topology import close_face
from tyssue.draw import sheet_view

## PLY Header

See also:

http://paulbourke.net/dataformats/ply/

```
ply
format ascii 1.0
element vertex 3090
property float x
property float y
property float z
element face 6871
property list int int vertex_index
property float area
property int epidermis
element edge 9928
property int source
property int target
property list int int face_index
property float length
element volume 32
property list int int face_index
property int label
end_header
```

In [5]:


"""
char       character                 1
uchar      unsigned character        1
short      short integer             2
ushort     unsigned short integer    2
int        integer                   4
uint       unsigned integer          4
float      single-precision float    4
double     double-precision float    8
"""

dtypes = {
    "char": np.uint8,
    "uchar": np.uint8,
    "short": np.int16,
    "ushort": np.uint16,
    "int": np.int32,
    "uint": np.uint32,
    "float": np.float32,
    "double": np.float64,
}

def _parse_header_line(line, element, structures, sizes):
    if line.startswith('element'):
        words = line.split(' ')
        element = words[1]
        structures[element] = {}
        sizes[element] = int(words[2]) 
        
    elif line.startswith('property'):
        words = line.split()
        if "list" in line:
            structures[element][words[-1]] = words[-3:-1]
        else:
            structures[element][words[-1]] = words[-2]
    return element


def parse_header(ply_f):
    structures, sizes = {}, {}
    element = ""
    sizes['header'] = 0
    for line in ply_f:
        sizes['header'] += 1
        if "end_header" in line:
            break
        element = _parse_header_line(
            line,
            element,
            structures, 
            sizes
        )
    else:
        raise IOError('Header end string not found')
    offsets = np.cumsum(list(sizes.values()))
    offsets = {k: v for k, v in zip(list(sizes)[1:], offsets[:-1])}
    
    return structures, sizes, offsets
    
def _parse_line(line, structure):
    words = line.split()
    cursor = 0
    values = {}
    for key, dtype in structure.items():
        if isinstance(dtype, str):
            dtype = dtypes[dtype]
            word = words[cursor]
            # special case for boolean values
            if word == 'True':
                values[key] = 1
            elif word == 'False':
                values[key] = 0
            else:
                values[key] = dtype(words[cursor])
            cursor += 1
        elif isinstance(dtype, list):
            dtype = dtypes[dtype[1]]
            length = int(words[cursor])
            cursor += 1
            values[key] = [dtype(w) for w in words[cursor: cursor+length]]
            cursor += length
    return values

def _parse_generic(ply_f, structure, size, offset):
    ply_f.seek(0)
    df = pd.DataFrame(columns=structure.keys())
    for i, line in enumerate(ply_f):
        if i < offset:
            continue
        if i == size + offset:
            break
        df = df.append(
            _parse_line(line, structure),
            ignore_index=True
        )
    return df.dropna(axis=1, how="all")
        
def _parse_vertex(ply_f, structure, size, offset):
    ply_f.seek(0)
    df = pd.read_csv(
        ply_f,
        nrows=size,
        header=None,
        sep=' ',
        skiprows=offset)
    df.columns = structure.keys()
    
    return df

def _parse_tri_face(ply_f, structure, size, offset):
    """
    Triangular faces
    element face 6871
    property list int int vertex_index
    property float area
    property int epidermis
    """
    ply_f.seek(0)
    df = pd.read_csv(
        ply_f,
        nrows=size,
        sep=' ',
        header=None,
        usecols=[1, 2, 3, 4, 5],
        skiprows=offset
    )
    df.columns = ["v0", "v1", "v2", "area", "epidermis"]
    
    return df


_parsers = {
    "vertex": _parse_vertex,
    "face": _parse_generic,
    "edge": _parse_generic,
    "volume": _parse_generic
}


def read_ply(ply_file):

    with open('data/ply/example_embryo3.ply', 'r', encoding='latin_1') as ply_f:
        structures, sizes, offsets = parse_header(ply_f)

        datasets = {}
        for element, structure in structures.items():

            parser = _parsers.get(element)
            if parser is None:
                continue
            size = sizes[element]
            offset = offsets[element]
            df = parser(ply_f, structure, size, offset)
            if element == "volume":
                datasets['cell'] = df
                df.index.name = "cell"
            else:
                datasets[element[:4]] = df
                df.index.name = element[:4]
            
    return datasets        
    


In [8]:


def load_ply(ply_file):
    """Reads a ply file and reconstructs the datasets
    necessary to construct an epithelium.
    
    """
    
    datasets = read_ply(ply_file)
    
    # repeat each edge to have one half-edge per face
    n_faces = datasets['edge']['face_index'].apply(len)
    edge_df_ = pd.DataFrame(
        np.repeat(
            datasets['edge'][['source', 'target']].to_numpy(),
            n_faces, axis=0),
        columns=['srce', 'trgt']
    )
    edge_df_['face_o'] = np.concatenate(datasets['edge']['face_index'])

    for col in datasets['edge'].columns:
        if not col in ('source', 'target', 'face_index'):
            edge_df_[col+"_o"] = np.repeat(datasets['edge'][col].to_numpy(), n_faces)

    edge_df_['edge_o'] = np.repeat(datasets['edge'].index.to_numpy(), n_faces)
    
    # repeat each face to have one half-face per cell
    n_faces = datasets['cell']['face_index'].apply(len)
    cell_faces = pd.DataFrame(
        np.repeat(datasets['cell'].index, n_faces),
        columns=['cell']
    )
    cell_faces['face'] = np.concatenate(datasets['cell']['face_index'])
    
    # for each edge, get the cells associated to its face
    edge_df_['cells'] = cell_faces.groupby("face")['cell'].apply(list).reindex(edge_df_['face_o']).to_numpy()
    
    # when no cell is attributed to a cell, put -1
    edge_df_['cells'] = edge_df_['cells'].apply(lambda df: [-1,] if not np.any(np.isfinite(df)) else df) 
    
    # repeat the edges to have an half-edge for each face and cell
    # This is the correct number of edges
    edge_df = pd.DataFrame(
        np.repeat(edge_df_.to_numpy(), edge_df_["cells"].apply(len), axis=0),
        columns = edge_df_.columns)

    edge_df['cell'] = np.concatenate(edge_df_["cells"].to_numpy())
    
    # clean up
    edge_df.drop(columns=['cells'], inplace=True)
    edge_df = edge_df.sort_values(['cell', "face_o"]).reset_index(drop=True)

    # face indices are still duplicated, we reindex the faces to avoid duplication
    # (frozensets are hashable so can be used as index)
    fc_pair = edge_df[["face_o", "cell"]].apply(frozenset, axis=1)

    ddup = fc_pair.drop_duplicates()
    new_faces = pd.Series(np.arange(ddup.shape[0]), index=ddup).reindex(fc_pair)
    edge_df['face'] = new_faces.to_numpy()

    cell_df = pd.DataFrame(index=edge_df['cell'].unique(), columns=list('xyz'))
    for col in datasets["cell"].columns:
        if col != "face_index":
            cell_df[col+"_o"] = datasets["cell"].loc[cell_df.index, col]
            
    face_df = pd.DataFrame(index=edge_df['face'].unique(), columns=list('xyz'))
    
    for col in datasets["face"].columns:
        if col != "vertex_index":
            edge_df["f_"+col] = datasets["face"][col].take(edge_df['face_o'].to_numpy()).to_numpy()
            face_df[col+"_o"] = edge_df.groupby("face")['f_'+col].first()
            edge_df.drop(columns=["f_"+col,], inplace=True)

    return {"vert": datasets["vert"], "face": face_df, "edge": edge_df, "cell": cell_df}


def get_eptm(ply_file):
    mono = Epithelium("from_ply", load_ply(ply_file))
    RNRGeometry.update_all(mono)

    inverted_edges = mono.edge_df[mono.edge_df['sub_vol'] <= 0].index

    mono.edge_df.loc[inverted_edges, ['srce', 'trgt']] = mono.edge_df.loc[inverted_edges, ['trgt', 'srce']].to_numpy()
    RNRGeometry.update_all(mono)
    RNRGeometry.center(mono)
    RNRGeometry.update_all(mono)

    mono.reset_index()
    mono.reset_topo()
    return mono


In [9]:

ply_file = 'data/ply/example_embryo3.ply'
mono = get_eptm(ply_file)

for face in mono.face_df.index:
    try:
        close_face(mono, face)
        RNRGeometry.update_all(mono)
    except ValueError:
        continue

mono.reset_index()
mono.reset_topo()

Closing only possible with exactly two dangling vertices
Closing only possible with exactly two dangling vertices
Closing only possible with exactly two dangling vertices
Closing only possible with exactly two dangling vertices
Closing only possible with exactly two dangling vertices


In [10]:
mono.face_df.head()

,x,y,z,area_o,epidermis_o,perimeter,area,num_sides
face,,,,,,,,
0,-24.776284,-37.130497,-1.407982,582.707642,1,154.457852,1639.660610,7
1,-6.956094,-40.271992,3.371077,21.059492,0,90.551518,338.997103,3
2,-21.134673,-29.320829,6.899434,383.691437,0,128.955852,1066.121715,6
3,-24.704027,-30.595317,-24.427965,33.309654,0,31.091506,32.325472,3
4,-36.375175,-18.853879,-4.274094,40.314571,0,57.609038,145.944825,4


In [12]:
ipv.clear()
mono.face_df['visible'] = True#mono.face_df['z'] > 0
fig, mesh = sheet_view(mono, mode='3D', face={'visible': True, "color": mono.face_df["area"]})
fig

Figure(camera=PerspectiveCamera(fov=46.0, position=(0.0, 0.0, 2.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.…

In [13]:
from tyssue.core.objects import _is_closed_cell
from tyssue.utils import single_cell


In [14]:
cells = mono.cell_df.index

for cell in cells:
    cell_ = single_cell(mono, cell)
    print(cell, cell_.validate())

0 True
1 False
2 True
3 True
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 True
14 True
15 False
16 False
17 True
18 True
19 False
20 True
21 True
22 True
23 True
24 False
25 False
26 False
27 False
28 False
29 False
30 True
31 False


In [15]:
ipv.clear()
cell_ = single_cell(mono, 29)
fig, mesh = sheet_view(cell_, mode="3D", face={"visible": True, "color": cell_.face_df["area"]})
fig

Figure(camera=PerspectiveCamera(fov=46.0, position=(0.0, 0.0, 2.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.…

In [16]:
ipv.clear()
fig, mesh = sheet_view(mono, mode="3D", face={"visible": True, "color": mono.face_df["area"]})
fig

Figure(camera=PerspectiveCamera(fov=46.0, position=(0.0, 0.0, 2.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.…